# IaC Maintainability Study - Analysis Notebook

This notebook loads experiment results from `/results/metrics/` and `/results/experiments/` and produces:
- Comparison tables across all 6 Terraform variants
- Coupling score and reuse ratio comparisons
- Drift susceptibility analysis by pattern
- Weighted maintainability scoring
- Tradeoff matrix visualization

In [ ]:
import json
import os
import glob
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np

# Configure display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)
pd.set_option('display.float_format', '{:.3f}'.format)

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

REPO_ROOT = Path('..').resolve().parent
METRICS_DIR = REPO_ROOT / 'results' / 'metrics'
EXPERIMENTS_DIR = REPO_ROOT / 'results' / 'experiments'
FIGURES_DIR = REPO_ROOT / 'paper' / 'figures'
FIGURES_DIR.mkdir(exist_ok=True)

print(f'Repository root: {REPO_ROOT}')
print(f'Metrics dir: {METRICS_DIR}')


In [ ]:
# ── Load structural metrics ───────────────────────────────────────────────────
def load_metrics(metrics_dir: Path) -> pd.DataFrame:
    records = []
    for json_file in sorted(metrics_dir.glob('*.json')):
        with open(json_file) as f:
            data = json.load(f)
        row = {'variant': data['variant']}
        row.update(data['metrics'])
        records.append(row)
    return pd.DataFrame(records)

# Fallback: use hardcoded results from paper if no results files found
PAPER_RESULTS = [
    {'variant': 'monolithic',       'total_loc': 1847, 'module_count': 0,  'coupling_score': 3.21, 'reuse_ratio': 0.00, 'resource_blocks': 48, 'module_calls': 0},
    {'variant': 'small_composable', 'total_loc': 2134, 'module_count': 6,  'coupling_score': 1.08, 'reuse_ratio': 0.87, 'resource_blocks': 8,  'module_calls': 6},
    {'variant': 'domain_based',     'total_loc': 1923, 'module_count': 3,  'coupling_score': 1.44, 'reuse_ratio': 0.76, 'resource_blocks': 12, 'module_calls': 3},
    {'variant': 'layer_based',      'total_loc': 2301, 'module_count': 5,  'coupling_score': 1.12, 'reuse_ratio': 0.82, 'resource_blocks': 10, 'module_calls': 5},
    {'variant': 'workspace_based',  'total_loc': 1891, 'module_count': 0,  'coupling_score': 3.18, 'reuse_ratio': 0.00, 'resource_blocks': 47, 'module_calls': 0},
    {'variant': 'state_per_stack',  'total_loc': 2287, 'module_count': 5,  'coupling_score': 0.94, 'reuse_ratio': 0.79, 'resource_blocks': 9,  'module_calls': 5},
]

if METRICS_DIR.exists() and list(METRICS_DIR.glob('*.json')):
    df_metrics = load_metrics(METRICS_DIR)
    print(f'Loaded {len(df_metrics)} metric files from results/')
else:
    print('No metric files found - using paper results for demonstration')
    df_metrics = pd.DataFrame(PAPER_RESULTS)

print(df_metrics.to_string(index=False))


In [ ]:
# ── Load experiment (drift) results ───────────────────────────────────────────
DRIFT_RESULTS = [
    {'variant': 'monolithic',       'scenario': '01_out_of_band',         'drift_detected': True,  'detection_latency_ms': 252, 'blast_radius': 52},
    {'variant': 'small_composable', 'scenario': '01_out_of_band',         'drift_detected': True,  'detection_latency_ms': 228, 'blast_radius': 8},
    {'variant': 'domain_based',     'scenario': '01_out_of_band',         'drift_detected': True,  'detection_latency_ms': 246, 'blast_radius': 19},
    {'variant': 'layer_based',      'scenario': '01_out_of_band',         'drift_detected': True,  'detection_latency_ms': 210, 'blast_radius': 9},
    {'variant': 'workspace_based',  'scenario': '01_out_of_band',         'drift_detected': True,  'detection_latency_ms': 258, 'blast_radius': 49},
    {'variant': 'state_per_stack',  'scenario': '01_out_of_band',         'drift_detected': True,  'detection_latency_ms': 192, 'blast_radius': 7},
    {'variant': 'monolithic',       'scenario': '02_version_drift',       'drift_detected': True,  'detection_latency_ms': 1840, 'blast_radius': 52},
    {'variant': 'small_composable', 'scenario': '02_version_drift',       'drift_detected': True,  'detection_latency_ms': 1820, 'blast_radius': 8},
    {'variant': 'monolithic',       'scenario': '03_provider_default',    'drift_detected': False, 'detection_latency_ms': 260,  'blast_radius': 0},
    {'variant': 'state_per_stack',  'scenario': '03_provider_default',    'drift_detected': True,  'detection_latency_ms': 198,  'blast_radius': 7},
    {'variant': 'monolithic',       'scenario': '06_iam_config_drift',    'drift_detected': True,  'detection_latency_ms': 310,  'blast_radius': 52},
    {'variant': 'state_per_stack',  'scenario': '06_iam_config_drift',    'drift_detected': True,  'detection_latency_ms': 205,  'blast_radius': 7},
]

def load_experiments(experiments_dir: Path) -> pd.DataFrame:
    records = []
    for exp_dir in sorted(experiments_dir.iterdir()):
        summary_file = exp_dir / 'experiment_summary.json'
        if summary_file.exists():
            with open(summary_file) as f:
                records.append(json.load(f))
    return pd.DataFrame(records) if records else pd.DataFrame()

if EXPERIMENTS_DIR.exists() and any(EXPERIMENTS_DIR.iterdir()):
    df_experiments = load_experiments(EXPERIMENTS_DIR)
    print(f'Loaded {len(df_experiments)} experiment results')
else:
    df_experiments = pd.DataFrame(DRIFT_RESULTS)
    print('Using demonstration drift results')

print(df_experiments.head(8).to_string(index=False))


In [ ]:
# ── Table 1: Structural Metrics Comparison ────────────────────────────────────
VARIANT_ORDER = ['monolithic', 'workspace_based', 'domain_based', 'small_composable', 'layer_based', 'state_per_stack']
VARIANT_LABELS = {
    'monolithic': 'Monolithic',
    'small_composable': 'Small Composable',
    'domain_based': 'Domain-Based',
    'layer_based': 'Layer-Based',
    'workspace_based': 'Workspace-Based',
    'state_per_stack': 'State-Per-Stack',
}

cols = ['variant', 'total_loc', 'module_count', 'coupling_score', 'reuse_ratio', 'resource_blocks']
available_cols = [c for c in cols if c in df_metrics.columns]
df_display = df_metrics[available_cols].copy()
df_display['variant_label'] = df_display['variant'].map(VARIANT_LABELS).fillna(df_display['variant'])

print('\n=== Table 1: Structural Metrics by Variant ===')
print(df_display.set_index('variant_label')[available_cols[1:]].to_string())


In [ ]:
# ── Figure 1: Coupling Score Bar Chart ───────────────────────────────────────
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

variants = df_metrics['variant'].tolist()
labels = [VARIANT_LABELS.get(v, v).replace(' ', '\n') for v in variants]
colors = ['#d62728' if c > 2.0 else '#ff7f0e' if c > 1.5 else '#2ca02c'
          for c in df_metrics.get('coupling_score', pd.Series([0]*len(variants)))]

ax1 = axes[0]
if 'coupling_score' in df_metrics.columns:
    bars = ax1.bar(labels, df_metrics['coupling_score'], color=colors, edgecolor='black', linewidth=0.5)
    ax1.axhline(y=1.5, color='orange', linestyle='--', alpha=0.7, label='High coupling threshold')
    ax1.axhline(y=2.5, color='red', linestyle='--', alpha=0.7, label='Critical coupling threshold')
    ax1.set_title('Coupling Score (E/N) by Variant', fontsize=13, fontweight='bold')
    ax1.set_ylabel('Coupling Score (lower = better)')
    ax1.set_ylim(0, max(df_metrics['coupling_score']) * 1.2)
    ax1.legend(fontsize=8)
    for bar, val in zip(bars, df_metrics['coupling_score']):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05,
                 f'{val:.2f}', ha='center', va='bottom', fontsize=9)

ax2 = axes[1]
if 'reuse_ratio' in df_metrics.columns:
    bars2 = ax2.bar(labels, df_metrics['reuse_ratio'], color='steelblue', edgecolor='black', linewidth=0.5)
    ax2.set_title('Reuse Ratio by Variant', fontsize=13, fontweight='bold')
    ax2.set_ylabel('Reuse Ratio (higher = better)')
    ax2.set_ylim(0, 1.1)
    for bar, val in zip(bars2, df_metrics['reuse_ratio']):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                 f'{val:.2f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
fig.savefig(FIGURES_DIR / 'fig1_coupling_reuse.png', dpi=150, bbox_inches='tight')
plt.show()
print(f'Figure saved to {FIGURES_DIR}/fig1_coupling_reuse.png')


In [ ]:
# ── Drift Susceptibility Analysis ────────────────────────────────────────────
if 'blast_radius' in df_experiments.columns and 'drift_detected' in df_experiments.columns:
    drift_summary = df_experiments.groupby('variant').agg(
        detection_rate=('drift_detected', 'mean'),
        mean_blast_radius=('blast_radius', 'mean'),
        mean_detection_latency_ms=('detection_latency_ms', 'mean'),
        experiment_count=('variant', 'count'),
    ).reset_index()
    drift_summary['detection_rate_pct'] = (drift_summary['detection_rate'] * 100).round(1)

    print('\n=== Table 2: Drift Detection Summary by Variant ===')
    print(drift_summary.to_string(index=False))


In [ ]:
# ── Figure 2: Blast Radius Comparison ─────────────────────────────────────────
if 'blast_radius' in df_experiments.columns:
    fig, ax = plt.subplots(figsize=(12, 5))

    for i, variant in enumerate(VARIANT_ORDER):
        subset = df_experiments[df_experiments['variant'] == variant]['blast_radius']
        if len(subset) > 0:
            ax.scatter([i] * len(subset), subset, alpha=0.7, s=80, zorder=5)
            ax.hlines(subset.mean(), i-0.3, i+0.3, colors='black', linewidth=2)

    ax.set_xticks(range(len(VARIANT_ORDER)))
    ax.set_xticklabels([VARIANT_LABELS.get(v, v).replace(' ', '\n') for v in VARIANT_ORDER])
    ax.set_title('Blast Radius by Variant (resources impacted per drift event)', fontsize=13, fontweight='bold')
    ax.set_ylabel('Resources in Plan Output (blast radius)')
    ax.set_yscale('log')

    plt.tight_layout()
    fig.savefig(FIGURES_DIR / 'fig2_blast_radius.png', dpi=150, bbox_inches='tight')
    plt.show()


In [ ]:
# ── Tradeoff Matrix ───────────────────────────────────────────────────────────
# Scoring: 5=best, 1=worst for each dimension
TRADEOFF_MATRIX = pd.DataFrame([
    {'variant': 'Monolithic',       'Initial Simplicity': 5, 'Coupling Isolation': 1, 'Change Surface': 1, 'Drift Blast Radius': 1, 'Operational Overhead': 5, 'Team Scalability': 1},
    {'variant': 'Small Composable', 'Initial Simplicity': 3, 'Coupling Isolation': 5, 'Change Surface': 5, 'Drift Blast Radius': 4, 'Operational Overhead': 3, 'Team Scalability': 4},
    {'variant': 'Domain-Based',     'Initial Simplicity': 4, 'Coupling Isolation': 3, 'Change Surface': 3, 'Drift Blast Radius': 3, 'Operational Overhead': 4, 'Team Scalability': 5},
    {'variant': 'Layer-Based',      'Initial Simplicity': 2, 'Coupling Isolation': 4, 'Change Surface': 5, 'Drift Blast Radius': 4, 'Operational Overhead': 2, 'Team Scalability': 3},
    {'variant': 'Workspace-Based',  'Initial Simplicity': 4, 'Coupling Isolation': 1, 'Change Surface': 1, 'Drift Blast Radius': 1, 'Operational Overhead': 5, 'Team Scalability': 2},
    {'variant': 'State-Per-Stack',  'Initial Simplicity': 1, 'Coupling Isolation': 5, 'Change Surface': 5, 'Drift Blast Radius': 5, 'Operational Overhead': 1, 'Team Scalability': 3},
]).set_index('variant')

print('\n=== Table 3: Tradeoff Matrix (5=best, 1=worst) ===')
print(TRADEOFF_MATRIX.to_string())

# Heatmap
fig, ax = plt.subplots(figsize=(12, 5))
im = ax.imshow(TRADEOFF_MATRIX.values, cmap='RdYlGn', vmin=1, vmax=5, aspect='auto')

ax.set_xticks(range(len(TRADEOFF_MATRIX.columns)))
ax.set_yticks(range(len(TRADEOFF_MATRIX.index)))
ax.set_xticklabels(TRADEOFF_MATRIX.columns, rotation=30, ha='right')
ax.set_yticklabels(TRADEOFF_MATRIX.index)

for i in range(len(TRADEOFF_MATRIX.index)):
    for j in range(len(TRADEOFF_MATRIX.columns)):
        ax.text(j, i, str(TRADEOFF_MATRIX.values[i, j]), ha='center', va='center',
                fontsize=12, fontweight='bold',
                color='white' if TRADEOFF_MATRIX.values[i, j] <= 2 else 'black')

plt.colorbar(im, ax=ax, label='Score (5=best, 1=worst)')
ax.set_title('IaC Pattern Tradeoff Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
fig.savefig(FIGURES_DIR / 'fig3_tradeoff_matrix.png', dpi=150, bbox_inches='tight')
plt.show()


In [ ]:
# ── Weighted Maintainability Score ────────────────────────────────────────────
# Weights reflect importance in production systems
WEIGHTS = {
    'Coupling Isolation':    0.30,
    'Change Surface':        0.25,
    'Drift Blast Radius':    0.25,
    'Team Scalability':      0.15,
    'Initial Simplicity':    0.05,
}
# Operational Overhead is excluded from weighted score (inverse relationship)

score_cols = list(WEIGHTS.keys())
weights_arr = np.array([WEIGHTS[c] for c in score_cols])
scores_arr = TRADEOFF_MATRIX[score_cols].values
weighted_scores = (scores_arr * weights_arr).sum(axis=1)

df_scores = pd.DataFrame({
    'variant': TRADEOFF_MATRIX.index,
    'weighted_maintainability_score': weighted_scores,
}).sort_values('weighted_maintainability_score', ascending=False)

print('\n=== Weighted Maintainability Scores ===')
print(df_scores.to_string(index=False))

fig, ax = plt.subplots(figsize=(10, 5))
bars = ax.barh(df_scores['variant'], df_scores['weighted_maintainability_score'],
               color=['gold' if s == df_scores['weighted_maintainability_score'].max() else 'steelblue'
                      for s in df_scores['weighted_maintainability_score']],
               edgecolor='black', linewidth=0.5)
ax.set_xlabel('Weighted Maintainability Score (0-5)')
ax.set_title('Overall Maintainability Score by Variant\n(Higher is better)', fontsize=13, fontweight='bold')
ax.set_xlim(0, 5.5)
for bar, val in zip(bars, df_scores['weighted_maintainability_score']):
    ax.text(val + 0.05, bar.get_y() + bar.get_height()/2,
            f'{val:.2f}', va='center', fontsize=10)

plt.tight_layout()
fig.savefig(FIGURES_DIR / 'fig4_maintainability_scores.png', dpi=150, bbox_inches='tight')
plt.show()


In [ ]:
# ── Pattern vs Drift Risk Matrix ──────────────────────────────────────────────
DRIFT_RISK = pd.DataFrame([
    {'pattern': 'Monolithic',       'Out-of-Band': 'HIGH',   'Version Drift': 'HIGH',   'Provider Default': 'HIGH',   'Data Source': 'MEDIUM', 'Partial Apply': 'HIGH',   'IAM Config': 'HIGH'},
    {'pattern': 'Small Composable', 'Out-of-Band': 'LOW',    'Version Drift': 'MEDIUM', 'Provider Default': 'MEDIUM', 'Data Source': 'LOW',    'Partial Apply': 'MEDIUM', 'IAM Config': 'LOW'},
    {'pattern': 'Domain-Based',     'Out-of-Band': 'MEDIUM', 'Version Drift': 'MEDIUM', 'Provider Default': 'MEDIUM', 'Data Source': 'MEDIUM', 'Partial Apply': 'MEDIUM', 'IAM Config': 'MEDIUM'},
    {'pattern': 'Layer-Based',      'Out-of-Band': 'LOW',    'Version Drift': 'LOW',    'Provider Default': 'LOW',    'Data Source': 'MEDIUM', 'Partial Apply': 'LOW',    'IAM Config': 'LOW'},
    {'pattern': 'Workspace-Based',  'Out-of-Band': 'HIGH',   'Version Drift': 'HIGH',   'Provider Default': 'HIGH',   'Data Source': 'MEDIUM', 'Partial Apply': 'HIGH',   'IAM Config': 'HIGH'},
    {'pattern': 'State-Per-Stack',  'Out-of-Band': 'LOW',    'Version Drift': 'LOW',    'Provider Default': 'LOW',    'Data Source': 'LOW',    'Partial Apply': 'LOW',    'IAM Config': 'LOW'},
]).set_index('pattern')

RISK_MAP = {'LOW': 1, 'MEDIUM': 2, 'HIGH': 3}
RISK_COLOR_MAP = {1: '#2ca02c', 2: '#ff7f0e', 3: '#d62728'}

risk_numeric = DRIFT_RISK.applymap(lambda x: RISK_MAP.get(x, 0))

fig, ax = plt.subplots(figsize=(14, 5))
im = ax.imshow(risk_numeric.values, cmap='RdYlGn_r', vmin=1, vmax=3, aspect='auto')

ax.set_xticks(range(len(DRIFT_RISK.columns)))
ax.set_yticks(range(len(DRIFT_RISK.index)))
ax.set_xticklabels(DRIFT_RISK.columns, rotation=30, ha='right')
ax.set_yticklabels(DRIFT_RISK.index)

for i in range(len(DRIFT_RISK.index)):
    for j in range(len(DRIFT_RISK.columns)):
        text = DRIFT_RISK.values[i, j]
        ax.text(j, i, text, ha='center', va='center', fontsize=10, fontweight='bold',
                color='white' if risk_numeric.values[i, j] == 3 else 'black')

legend_handles = [
    mpatches.Patch(color='#2ca02c', label='LOW'),
    mpatches.Patch(color='#ff7f0e', label='MEDIUM'),
    mpatches.Patch(color='#d62728', label='HIGH'),
]
ax.legend(handles=legend_handles, loc='upper right', bbox_to_anchor=(1.15, 1))
ax.set_title('Drift Risk by Pattern and Scenario', fontsize=14, fontweight='bold')
plt.tight_layout()
fig.savefig(FIGURES_DIR / 'fig5_drift_risk_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

print('\nAll figures saved to', FIGURES_DIR)
